In [1]:
from aocd import get_data

Given a list of calibration equations, determine which ones can be made true by inserting `+` (add) or `*` (multiply) operators between the numbers. Operators are evaluated left-to-right (not by precedence), and numbers cannot be rearranged.

Each line represents an equation in the format:

`target: num1 num2 num3 ...`

`target` is the test value you need to match.

`num1` `num2` `num3` ... are the numbers to combine using `+` and `*`.

The final solution should return the sum of all `target` values for equations that can be made true by inserting `+` or `*`.

In [2]:
example_input = """
190: 10 19  
3267: 81 40 27  
83: 17 5  
156: 15 6  
7290: 6 8 6 15  
161011: 16 10 13  
192: 17 8 14  
21037: 9 7 18 13  
292: 11 6 16 20  
"""

For the above example, the solution would be `3749`

Explanation:
From the example:

`190`: `10 19 → 10 * 19 = 190` ✅

`3267`: `81 40 27 → 81 + 40 * 27 = 3267` ✅

`292`: `11 6 16 20 → 11 + 6 * 16 + 20 = 292` ✅

Other equations cannot match their target with any combination of `+` or `*`.

The sum of valid targets: `190 + 3267 + 292 = 3749`.

What we need to do:

- Generate all possible combinations of operators given input length
- Insert `+` or `*` between numbers using all possible combinations
- Evaluate the expressions left-to-right
- Check if any combination equals the target
- If valid, add the target to the result
- Return the sum of all valid targets

First generate all the possible combinations that could be used for the numbers

In [3]:
from itertools import product

symbols = ['*', '+']

def generate_combinations(n):
    # return set of tuples
    return set(product(symbols, repeat=n))

# Example for 3 elements
combinations = generate_combinations(3)
for combo in combinations:
    print(combo)

('+', '+', '*')
('*', '*', '+')
('*', '*', '*')
('+', '+', '+')
('*', '+', '+')
('*', '+', '*')
('+', '*', '+')
('+', '*', '*')


In [4]:
import re

def parse_data(input_str):
    return [(int(t), list(map(int, n.split()))) for t, n in re.findall(r"(\d+):\s*([\d\s]+)\n", input_str)]

In [5]:
def left_to_right_eval(numbers, ops):
    result = numbers[0]
    
    for i in range(len(ops)):
        operator = ops[i]
        number = numbers[i + 1]
        
        if operator == '+':
            result += number
        elif operator == '*':
            result *= number
    
    return result

Wrote recursively for fun / interview practice. Despite recursion not actually being great here:

In [6]:
def left_to_right_eval_recur(numbers, ops):
    if not ops:
        return numbers[0]

    if ops[0] == '+':
        result = numbers[0] + numbers[1]
    elif ops[0] == '*':
        result = numbers[0] * numbers[1]

    return left_to_right_eval_recur([result] + numbers[2:], ops[1:])

Recursion causes a performance overhead!

*Function Call Overhead:* Each recursive call adds a new frame to the call stack. This increases memory usage and function call time, making recursion less efficient than iteration for large datasets.

*Stack Limits:* Python has a recursion depth limit (default is typically 1000). For large inputs, the recursive function can result in a RecursionError.

In [7]:
total = 0
for target, numbers in parse_data(example_input):
    
    operator_combinations = generate_combinations(len(numbers) - 1)
    
    for ops in operator_combinations:
        result = left_to_right_eval_recur(numbers, ops)
        
        if result == target:
            print(f'{target}: {numbers} {ops}')
            total += target
            break  # we only need the first solution  
        

190: [10, 19] ('*',)
3267: [81, 40, 27] ('+', '*')
292: [11, 6, 16, 20] ('+', '*', '+')


In [8]:
total

3749

In [9]:
data = get_data(day=7, year=2024)

In [10]:
total = 0
for target, numbers in parse_data(data):
    
    operator_combinations = generate_combinations(len(numbers) - 1)
    
    for ops in operator_combinations:
        result = left_to_right_eval(numbers, ops)
        
        if result == target:
            total += target
            break  # we only need the first solution  

In [11]:
total

2941973819040

## Part Two

Determine which equations can be made true by introducing a new operator: concatenation (`||`)

- Update `generate_combinations` to take an array of symbols instead of having them hardcoded
- Update our evaluation function to include an operator for concatenation

In [12]:
new_symbols = ['*', '+', '||']

def generate_combinations(symbols, n):    
    return set(product(symbols, repeat=n))

In [13]:
def left_to_right_eval(numbers, ops):
    result = numbers[0]
    
    for i in range(len(ops)):
        operator = ops[i]
        number = numbers[i + 1]
        
        if operator == '+':
            result += number
        elif operator == '*':
            result *= number
        elif operator == '||':
            result = int(f'{result}{number}')
    
    return result

We can replace `if-elif-...` with a dictionary

In [14]:
evaluate = {
    '+': lambda x: result + x,
    '*': lambda x: result * x,
    '||': lambda x: int(f'{result}{number}')
}

def left_to_right_eval_new(numbers, ops):
    result = numbers[0]
    
    for i in range(len(ops)):
        operator = ops[i]
        number = numbers[i + 1]

        result = evaluate[operator](number)
    
    return result

In [17]:
def search_solution(target, numbers, operator_combinations):
    for ops in operator_combinations:
        result = left_to_right_eval(numbers, ops)
        if result == target:
            return target

In [18]:
total = 0

for target, numbers in parse_data(data):

    operator_combinations = generate_combinations(symbols, len(numbers) - 1)
    if result := search_solution(target, numbers, operator_combinations):
        total += target
        continue  # move onto the next target

    # no point in trying the basic combinations again
    additional_combinations = generate_combinations(new_symbols, len(numbers) - 1) - operator_combinations
    if result := search_solution(target, numbers, additional_combinations):
        total += result

In [19]:
print(total)

249943037374011
